In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install tensorflow==1.12.0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 25287, done.
remote: Total 25287 (delta 0), reused 0 (delta 0), pack-reused 25287
Receiving objects: 100% (25287/25287), 507.67 MiB | 30.87 MiB/s, done.
Resolving deltas: 100% (15114/15114), done.
Checking out files: 100% (2868/2868), done.


In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
!get_ipython().system_raw(‘tensorboard --logdir /content/models/research --host 0.0.0.0 --port 6006 &’)

/bin/bash: -c: line 0: syntax error near unexpected token `.system_raw'
/bin/bash: -c: line 0: `get_ipython().system_raw(‘tensorboard --logdir /content/models/research --host 0.0.0.0 --port 6006 &’)'


### **upload protobuf.zip**

In [0]:
!mkdir object-detection

In [0]:
!rsync -r /content/drive/My\ Drive/data_colab/  /content/object-detection


In [0]:
%cd object-detection

/content/object-detection


In [0]:
!ls
!mv protobuf.zip /content/models/research
%cd /content/models/research
!unzip protobuf.zip

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

env: PYTHONPATH=/content/models/research:/content/models/research/slim


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
!python object_detection/builders/model_builder_test.py

................
----------------------------------------------------------------------
Ran 16 tests in 0.062s

OK


In [0]:
%cd  
!ls
%cd /content/
!ls

/root
content  data
/content
adc.json  drive   models  ngrok-stable-linux-amd64.zip	sample_data
data	  gdrive  ngrok   object-detection


In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz

--2019-04-20 13:38:31--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128048406 (122M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz’

ssd_mobilenet_v1_co 100%[===================>] 122.12M   168MB/s    in 0.7s    

2019-04-20 13:38:31 (168 MB/s) - ‘ssd_mobilenet_v1_coco_11_06_2017.tar.gz’ saved [128048406/128048406]



## **upload images.zip**

In [0]:
!ls
%cd object-detection
!unzip images.zip

In [0]:
%cd /content/object-detection 
!ls
!mv images /content/models/research/object_detection
!rsync -r data/ /content/models/research/object_detection/data
!mv training /content/models/research/object_detection/training

In [0]:
%cd /content/models/research/
!python3 setup.py install

In [0]:
!mv generate_tfrecord.py object-detection/

In [0]:
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_dir=images/train

# New Section

In [0]:
!mv object-detection.pbtxt  /content/models/research/object_detection/data


In [0]:
%cd /content/models/research/object_detection
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
!tar -xvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

In [0]:
!python3 legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` mod

In [0]:
!python3 export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path training/ssd_mobilenet_v1_pets.config \
    --trained_checkpoint_prefix training/model.ckpt-580 \
    --output_directory hand_sign_out_inference_graph

In [0]:
!rsync -r  /content/object-detection/hand_sign_out_inference_graph  /content/drive/My\ Drive/data_colab/hand_sign_out_inference_graph

In [0]:
!ls

gdrive	sample_data
